## Import

In [1]:
%run indicator_functions.ipynb

import pandas as pd
import numpy as np

import requests
import json

import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import display
pd.set_option('display.max_columns', None)

### API Call

In [2]:
df = AV_daily_call('AMZN', output_size='full')

df.index = pd.to_datetime(df.index)

In [3]:
df

,Open,High,Low,Close,Volume
AMZN,,,,,
1999-11-01,68.06,71.8800,66.3100,69.13,12824100.0
1999-11-02,69.75,70.0000,65.0600,66.44,13243200.0
1999-11-03,68.19,68.5000,65.0000,65.81,10772100.0
1999-11-04,67.19,67.1900,61.0000,63.06,16759200.0
1999-11-05,64.75,65.5000,62.2500,64.94,11091400.0
...,...,...,...,...,...
2021-02-10,3314.00,3317.9548,3254.0000,3286.58,3125174.0
2021-02-11,3292.00,3292.0000,3248.0600,3262.13,2301417.0
2021-02-12,3250.00,3280.2500,3233.3100,3277.71,2335339.0


## Column Engineering

### Sentiment

In [4]:
df['r_percent_increase_high'] = np.nan
df['r_percent_increase_close'] = np.nan
df['c_four_percent_high'] = np.nan


for i in range(len(df)):
    
    if i == len(df)-5:
        break
        
    current_close = df.iloc[i]['Close']
    max_high = max(df.iloc[i+1 : i+6]['High'])
    max_close = max(df.iloc[i+1 : i+6]['Close'])
    
    percent_change_up = (max_high - current_close) / current_close
    percent_change_up_close = (max_close - current_close) / current_close
    
    df.iloc[i, df.columns.get_loc('r_percent_increase_high')] = percent_change_up * 100
    df.iloc[i, df.columns.get_loc('r_percent_increase_close')] = percent_change_up_close * 100
    
    if percent_change_up >= .04:
        df.iloc[i, df.columns.get_loc('c_four_percent_high')] = 'Buy'
    else:
        df.iloc[i, df.columns.get_loc('c_four_percent_high')] = '0'

### Technical Indicators

In [5]:
df.iloc[-505 : -495]

,Open,High,Low,Close,Volume,r_percent_increase_high,r_percent_increase_close,c_four_percent_high
AMZN,,,,,,,,
2019-02-15,1627.86,1628.91,1604.50,1607.95,4343893.0,2.901210,1.557884,0
2019-02-19,1601.00,1634.00,1600.56,1627.58,3681656.0,1.660133,0.541909,0
2019-02-20,1630.00,1634.93,1610.12,1622.10,3337589.0,2.003576,1.170705,0
2019-02-21,1619.85,1623.56,1600.91,1619.44,3483392.0,2.171121,1.336882,0
2019-02-22,1623.50,1634.94,1621.17,1631.56,3096191.0,2.617127,2.462061,0
2019-02-25,1641.45,1654.60,1630.39,1633.00,3184462.0,4.680343,3.868340,Buy
2019-02-26,1625.98,1639.99,1616.13,1636.40,2665815.0,4.462845,3.652530,Buy
2019-02-27,1628.18,1641.81,1615.10,1641.09,3148824.0,4.164305,3.356306,Buy
2019-02-28,1635.25,1651.77,1633.83,1639.83,3025891.0,4.244342,3.435722,Buy


In [6]:
for i in range(len(df)):
    sma(df, i)
    stochastic(df, i)
    rsi(df,i)
    roc(df,i)
    atr(df,i)
    adx(df,i)

## Binning

In [7]:
df['rsi_over_80'] = [1 if x >=80 else 0 for x in df['RSI']]
df['rsi_over_70'] = [1 if x >=70 else 0 for x in df['RSI']]
df['rsi_over_60'] = [1 if x >=60 else 0 for x in df['RSI']]

df['rsi_under_40'] = [1 if x <=40 else 0 for x in df['RSI']]
df['rsi_under_30'] = [1 if x <=30 else 0 for x in df['RSI']]
df['rsi_under_20'] = [1 if x <=30 else 0 for x in df['RSI']]

df['sto_over_80'] = [1 if x >=80 else 0 for x in df['Stochastic']]
df['sto_over_70'] = [1 if x >=70 else 0 for x in df['Stochastic']]
df['sto_over_60'] = [1 if x >=60 else 0 for x in df['Stochastic']]

df['sto_under_40'] = [1 if x <=40 else 0 for x in df['Stochastic']]
df['sto_under_30'] = [1 if x <=30 else 0 for x in df['Stochastic']]
df['sto_under_20'] = [1 if x <=30 else 0 for x in df['Stochastic']]

## Export

In [8]:
df.tail(10)

,Open,High,Low,Close,Volume,r_percent_increase_high,r_percent_increase_close,c_four_percent_high,SMA,SMA_diff,Stochastic,Stochastic_diff,rsi_up_var,rsi_down_var,RSI,RSI_diff,ROC,ROC_diff,tr,ATR,ATR_diff,pos_dx,neg_dx,dx,ADX,ADX_diff,rsi_over_80,rsi_over_70,rsi_over_60,rsi_under_40,rsi_under_30,rsi_under_20,sto_over_80,sto_over_70,sto_over_60,sto_under_40,sto_under_30,sto_under_20
AMZN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-02-03,3425.01,3434.0000,3308.6200,3312.53,7088781.0,1.946247,1.196065,0,3291.574444,0.615556,64.150164,-21.494964,0.00,67.47,60.714286,-8.863149,0.167524,-3.406072,125.3800,88.029586,4.018571,6.26,0.00,0.867212,17.325803,-0.145051,0,0,1,0,0,0,0,0,1,0,0,0
2021-02-04,3330.00,3347.0000,3277.7500,3331.00,3670661.0,1.380967,0.634944,0,3295.882222,4.307778,69.601275,5.451111,18.47,0.00,65.317519,4.603233,1.177621,1.010097,69.2500,88.875307,0.845721,0.00,0.00,0.867212,17.273923,-0.051880,0,0,1,0,0,0,0,0,1,0,0,0
2021-02-05,3319.00,3377.0000,3302.7100,3352.15,3620808.0,0.383333,-0.871381,0,3302.343333,6.461111,75.784024,6.182749,21.15,0.00,68.715084,3.397565,1.765331,0.587710,74.2900,90.797807,1.922500,30.00,0.00,5.519249,17.554331,0.280408,0,0,1,0,0,0,0,1,1,0,0,0
2021-02-08,3358.50,3364.9999,3304.0000,3322.94,3257391.0,0.453213,-0.539883,0,3301.988889,-0.354444,57.119691,-18.664333,0.00,29.21,64.976296,-3.738788,-0.095907,-1.861238,60.9999,91.654943,0.857136,0.00,1.29,4.942447,14.268481,-3.285851,0,0,1,0,0,0,0,0,0,0,0,0
2021-02-09,3312.49,3338.0000,3297.8400,3305.00,2179355.0,0.481392,0.110136,0,3310.035556,8.046667,48.286230,-8.833461,0.00,17.94,53.781436,-11.194860,2.240316,2.336223,40.1600,83.163514,-8.491429,0.00,0.00,20.515965,11.441331,-2.827150,0,0,0,0,0,0,0,0,0,0,0,0
2021-02-10,3314.00,3317.9548,3254.0000,3286.58,3125174.0,NaN,NaN,NaN,3315.475556,5.440000,40.901984,-7.384245,0.00,18.42,48.056672,-5.724765,1.512222,-0.728094,63.9548,81.648143,-1.515371,0.00,0.00,2.908345,11.117971,-0.323360,0,0,0,0,0,0,0,0,0,0,0,0
2021-02-11,3292.00,3292.0000,3248.0600,3262.13,2301417.0,NaN,NaN,NaN,3321.690000,6.214444,31.100421,-9.801563,0.00,24.45,47.185969,-0.870703,1.744433,0.232211,43.9400,82.019171,0.371029,0.00,0.00,2.908345,10.794611,-0.323360,0,0,0,0,0,0,0,0,0,1,0,0
2021-02-12,3250.00,3280.2500,3233.3100,3277.71,2335339.0,NaN,NaN,NaN,3314.448889,-7.241111,37.346162,6.245741,15.58,0.00,48.515393,1.329424,-1.949517,-3.693949,46.9400,76.747479,-5.271693,0.00,0.00,8.845452,10.177171,-0.617440,0,0,0,0,0,0,0,0,0,1,0,0
2021-02-16,3254.05,3308.3000,3253.5910,3268.95,2574721.0,NaN,NaN,NaN,3302.110000,-12.338889,33.834436,-3.511726,0.00,8.76,44.556981,-3.958412,-3.285503,-1.335986,54.7090,76.717407,-0.030071,28.05,0.00,11.071322,10.105706,-0.071465,0,0,0,0,0,0,0,0,0,1,0,0


In [9]:
df.to_csv('data/Cleaned_amzn_df.csv')